In [9]:
import os
import json
import re
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


def replace_in_file(file_path, rename_map):
    """Replace occurrences in a file using regex with word boundaries."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        original_content = content
        # Create a regex pattern that matches any of the keys with word boundaries
        pattern = r'\b(?:' + '|'.join(re.escape(key) for key in rename_map.keys()) + r')\b'

        def replacer(match):
            return rename_map[match.group(0)]

        new_content = re.sub(pattern, replacer, content)

        # Only write if content changed
        if new_content != original_content:
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)
            return True
        return False
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return False


def process_directory(directory, rename_map):
    """Process all Python files in directory using multiple threads."""
    py_files = [os.path.join(root, file)
                for root, _, files in os.walk(directory)
                for file in files if file.endswith('.py')]

    if not py_files:
        print("No Python files found in the directory.")
        return

    results = [False] * len(py_files)

    # Function to be executed by each thread
    def process_file(index, file_path):
        results[index] = replace_in_file(file_path, rename_map)

    # Create and start threads with progress bar
    with tqdm(total=len(py_files), desc="Replacing variables/functions", unit="file") as pbar:
        with ThreadPoolExecutor(max_workers=os.cpu_count() or 4) as executor:
            # Submit tasks and keep track of futures
            futures = []
            for i, file_path in enumerate(py_files):
                future = executor.submit(process_file, i, file_path)
                future.add_done_callback(lambda p: pbar.update(1))
                futures.append(future)

            # Wait for all tasks to complete
            for future in futures:
                try:
                    future.result()
                except Exception as e:
                    print(f"Thread error: {e}")

    changed_files = sum(results)
    print(f"Completed! Changed {changed_files} of {len(py_files)} files.")


if __name__ == "__main__":
    project_dir = input("Enter your project directory: ")
    map_file = "rename_map.json"

    with open(map_file, 'r', encoding='utf-8') as f:
        rename_map = json.load(f)

    process_directory(project_dir, rename_map)

Replacing variables/functions: 100%|██████████| 143/143 [00:02<00:00, 53.36file/s] 

Completed! Changed 90 of 143 files.
